<a href="https://colab.research.google.com/github/motazsaad/infant-cry-care/blob/main/CryCareDraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
title = 'CryCareSheet'
spreedSheet = gc.open(title)
audioContSheet = spreedSheet.sheet1

In [ ]:
!pip install ffmpeg
!pip install split-folders
!pip install git+https://github.com/jiaaro/pydub.git@master

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=1dc00ec92d91376b657934e4057a355fc17b499b707a77b68f5b640ba258b85e
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
  Cloning https://github.com/jiaaro/pydub.git (to revision master) to /tmp/pip-req-build-a5vkt0x_
  Running command git clone -q https://github.com/jiaaro/pydub.git /tmp/pip-req-build-a5vkt0x_
  Created wheel for pydub: filename=pydub-0.25.1-py2.py3-none-any.whl size=32352 sha256=ba8e9a0f13b5a9015c967f5ba2e92de711964563bdd877ac7ab6f2e8312d71de
  Stored in directory: /tmp/pip-ephem-wheel-cache-7n0bpgeg/wheels/a7/e5/a1/626333b98a5e043f69b7bdd3a989b87b8ccf9005f4b66a6aef
Successfully built pydub


In [ ]:
import glob
import subprocess
import os
from pydub import AudioSegment
from pydub.utils import make_chunks
from pydub.silence import split_on_silence
import pandas as pd
import librosa 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from gspread_dataframe import set_with_dataframe
import IPython.display as ipd


mainPath = '/content/drive/MyDrive/CryCareDataset/'
originDataPath = mainPath + 'Dataset/'
oneSecDataPath = mainPath + 'oneSecDataset/'
onSilencePath = mainPath + 'onSilence/'
donateACryPath = mainPath + 'donateacry/'
generatedData = mainPath + 'generatedData/'
exp1Path = mainPath + 'generatedData/Exp1/'
exp2Path = mainPath + 'generatedData/Exp2/'
exp3Path = mainPath + 'generatedData/Exp3/'
exp4Path = mainPath + 'generatedData/Exp4/'
exp5Path = mainPath + 'generatedData/Exp5/'
exp6Path = mainPath + 'generatedData/Exp6/'
exp7Path = mainPath + 'generatedData/Exp7/'

directories = ['Hungry', 'lower gas', 'burp up', 'Sleepy', 'Uncomfortable'] 


In [ ]:
def makeDirectory(newPath, directories):
  try:
    os.mkdir(newPath)
    for directoryName in directories:
      os.mkdir(newPath + directoryName)
  except:
    print('File exists')

#make a new directories in drive for oneSecDataset
# makeDirectory(oneSecDataPath, directories)
# makeDirectory(onSilencePath, directories)
# makeDirectory(generatedData, directories)
# makeDirectory(generatedData2, directories)


# makeDirectory(exp1Path, directories)
# makeDirectory(exp2Path, directories)
# makeDirectory(exp3Path, directories)
makeDirectory(exp4Path, directories)
# makeDirectory(exp5Path, directories)
# makeDirectory(exp6Path, directories)
# makeDirectory(exp7Path, directories)

In [ ]:
def copyOriginalData (path):
  for directory in directories:
    for audiofile in glob.glob(originDataPath + directory + '/**.wav'):
      filename = audiofile.split('/')[-1]
      newPath = path + directory + '/' + filename
      subprocess.call(['ffmpeg', '-i', audiofile,
                      newPath ])
# copyOriginalData(generatedData)
# copyOriginalData(exp1Path)
# copyOriginalData(exp2Path)
# copyOriginalData(exp3Path)
copyOriginalData(exp4Path)
# copyOriginalData(exp5Path)
# copyOriginalData(exp6Path)
# copyOriginalData(exp7Path)


In [ ]:
def noise_injection(data, noise_factor):
    data, sr = librosa.load(data, sr = 11025)
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise

    # # Cast back to same data type
    augmented_data = augmented_data.astype(type(data[0]))
    return augmented_data

In [ ]:
def shift_time(data,shift_direction):
    data, sr = librosa.load(data, sr = 11025)

    shift = np.random.randint(11025 * 1)
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
                
    augmented_data = np.roll(data, shift)

    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
   
    return augmented_data

In [ ]:
def change_speed(data, speed_factor):
    data, sr = librosa.load(data, sr = 11025)
    return librosa.effects.time_stretch(data, speed_factor)

In [ ]:
def pitch_shift(data, factor):
    data, sr = librosa.load(data, sr = 11025)
    return librosa.effects.pitch_shift(data, 11025, factor)

In [ ]:
def dataAugmentation(path, directories, type, speedFactor = 0.65, pitchFactor = 1) :
  print(speedFactor, pitchFactor)
  for directory in directories:
    for audiofile in glob.glob(originDataPath + directory + '/**.wav'):
      filewithoutext = audiofile.split('/')[-1]
      file_name = filewithoutext.split('.')[0] + '_' +type +'.wav'
      new_path = path + directory + '/' + file_name
      if(type == 'noise'):
        new_audio = noise_injection(audiofile, 0.003)
      elif(type == 'speed'):
        new_audio = change_speed(audiofile, speedFactor)
      elif (type == 'pitch'):
        new_audio = pitch_shift(audiofile, pitchFactor)
      elif (type == 'shift'):
        new_audio = shift_time(audiofile, 'both')
      try:
        new_audio = ipd.Audio(new_audio, rate=11025)
        new_audio = AudioSegment(new_audio.data, frame_rate=11025, sample_width=2, channels=1)
        new_audio.export( new_path , format="wav")
      except:
        pass

# dataAugmentation(directories, 'speed')
# dataAugmentation(directories, 'noise')
# dataAugmentation(directories, 'pitch')
# dataAugmentation(directories, 'shift')
directoriesExp = ['lower gas', 'burp up', 'Sleepy', 'Uncomfortable']
def ExpSpeed (path, speedFactor, pitchFactor):
  dataAugmentation(path, directoriesExp , 'speed', speedFactor)
  dataAugmentation(path, directoriesExp , 'noise')
  dataAugmentation(path, directoriesExp , 'shift')
  dataAugmentation(path, directoriesExp , 'pitch', speedFactor, pitchFactor)

# ExpSpeed(exp1Path,0.65)
# ExpSpeed(exp2Path,0.8)
# ExpSpeed(exp3Path,1.1)
# ExpSpeed(exp5Path,1.1,0.3)
# ExpSpeed(exp6Path,1.1,0.5)
ExpSpeed(exp4Path,1.1,1)

1.1 1
0.65 1
0.65 1


/usr/local/lib/python3.7/dist-packages/IPython/lib/display.py:135: RuntimeWarning: invalid value encountered in true_divide
  scaled = np.int16(data/np.max(np.abs(data))*32767).tolist()


1.1 1


In [ ]:
audioPath = originDataPath + 'Hungry/7.wav'
new = noise_injection(audioPath, 0)
audio = ipd.Audio(new, rate=11025)
audio

In [ ]:
new = shift_time(audioPath, 'both')
audio = ipd.Audio(new, rate=11025)
audio

In [ ]:
audioPath = originDataPath + 'Hungry/6.wav'
new = pitch_shift(audioPath, 0.3)
audio = ipd.Audio(new, rate=11025)
audio

In [ ]:
audioPath = originDataPath + 'Hungry/6.wav'
new = change_speed(audioPath, 1.3)
audio = ipd.Audio(new, rate=11025)
audio

In [ ]:
import splitfolders
def splitFolders(path):
  splitfolders.ratio(path, output= path + "output",  ratio=(.75,.25), group_prefix=None) 
# splitFolders(originDataPath)
# splitFolders(oneSecDataPath)
# splitFolders(onSilencePath)
# splitFolders(generatedData)
# splitFolders(generatedData2)

# splitFolders(exp1Path)
# splitFolders(exp2Path)
# splitFolders(exp3Path)
splitFolders(exp4Path)
# splitFolders(exp5Path)
# splitFolders(exp6Path)
# splitFolders(exp7Path)

Copying files: 2690 files [00:30, 89.48 files/s]


In [ ]:
def filterAudioFile ():
  for directory in os.listdir(donateACryPath):
    for file in glob.glob(donateACryPath + directory + '/**.wav'):
      age = file.split('-')[-2]
      filename = file.split('/')[-1]
      newPath = originDataPath + directory + '/' + filename
      if(age == '04' or age == '48' or age == '26'):
        subprocess.call(['ffmpeg', '-i', file,
                      newPath])
filterAudioFile()

In [ ]:
def convertMp3ToWav(path, directories) :
  for directory in directories:
    for filename in glob.glob(path + directory + '/**.mp3'):
        filewithoutext = filename.split('.')
        wavfile = filewithoutext[0] + '.wav'
        subprocess.call(['ffmpeg', '-i', filename,
                      wavfile])
        os.remove(filename)
      
#convert all directories to wav
convertMp3ToWav(originDataPath, directories)

In [ ]:
#one sec chunks

#Export all of the individual chunks as wav files
def exportChunks(path, directory, filename, chunks):
  # for i, chunk in enumerate(myaudio[::1000]):
  for i, chunk in enumerate(chunks):
    filewithoutext = filename.split('/')
    name = filewithoutext[-1].split('.')[0]
    chunk_name = name + "chunk{0}.wav".format(i)
    # if(len(chunk) > 900): 
    chunk.export(path + directory + '/' + chunk_name, format="wav")

def makeChunks(path, directories) :
  for directoryName in directories:
    for filename in glob.glob(path + directoryName + '/**.wav'):
      myaudio = AudioSegment.from_file(filename , "wav") 
      chunk_length_ms = 1000 # pydub calculates in millisec
      chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
      exportChunks(oneSecDataPath, directoryName, filename, chunks)

# makeChunks(originDataPath, directories)


In [ ]:
def makeChunksOnSilence(path, directories):
  for directoryName in directories:
    for filename in glob.glob(path + directoryName + '/**.wav'):
      myaudio = AudioSegment.from_file(filename , "wav") 
      myaudio = myaudio.set_frame_rate(11025)
      # print(len(myaudio), filename)
      chunks = split_on_silence(myaudio, silence_thresh=-40, min_silence_len=500)
      exportChunks(onSilencePath, directoryName, filename, chunks)

makeChunksOnSilence(generatedData, directories)

In [ ]:
#Store all audio files in dictionary where key: filename, value: label
def createRawAudio(path) :
  print(path)
  raw_audio = dict()
  for directory in directories :
    for filename in glob.glob(path + directory + '/**.wav'):
      raw_audio[filename] = directory
  return raw_audio

In [ ]:
def audioFeatureExtraction(raw_audio, featureExtraction):
  X = pd.DataFrame(columns = np.arange(45), dtype = 'float32').astype(np.float32)
  for i, filename in enumerate(raw_audio):
      if filename.endswith(".wav"):
          audiofile, sr = librosa.load(filename, sr = 11025)
          fingerprint = featureExtraction(y=audiofile, sr=sr)
          x = pd.DataFrame(fingerprint, dtype = 'float32')
          x[44] = raw_audio[filename]
          X.loc[i] = x.loc[0]
  X = X.fillna(0)
  y = X[44]
  del X[44]
  X = X.astype(np.float32)
  return X, y

In [ ]:
def trainTestSplit(X):
  y = X[44]
  del X[44]
  X = X.astype(np.float32)
  X_train, X_test, y_train, y_test = train_test_split(X, y)
  return X_train, X_test, y_train, y_test

In [ ]:
def get_scores(classifier, X_train, X_test, y_train, y_test, **kwargs):
        model = classifier(**kwargs)
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        return classification_report(y_test, y_predict, target_names=directories, output_dict=True)
# randomForest =  get_scores(RandomForestClassifier, X_train, X_test, y_train, y_test)
# SVM =  get_scores(SVC, X_train, X_test, y_train, y_test)


In [ ]:
def exportClassificationReport(report, sheetName):
  df = pd.DataFrame(report).transpose()
  df[''] = directories + ['accuracy', 'macro avg', 'weighted avg']
  try:
    spreedSheet.add_worksheet(title =sheetName, rows = 200, cols = 100)
  except:
    print('file exist')
  reportSheet = spreedSheet.worksheet(sheetName)
  set_with_dataframe(reportSheet, df) 
  # return df

In [ ]:
df = pd.DataFrame()
df[''] = directories
def countDirectories(path, dataName):
  audio_count = []
  for directoryName in directories:
    count = 0
    for filename in glob.glob(path + directoryName + '/**.wav'):
      count +=1
    audio_count.append(count)
    count = 0
  df[dataName] = audio_count

countDirectories(originDataPath, 'originData')
countDirectories(onSilencePath, 'onSilence')
countDirectories(oneSecDataPath, 'oneSec')
countDirectories(generatedData, 'generatedData')
countDirectories(exp1Path, 'exp1Path')
countDirectories(exp2Path, 'exp2Path')
countDirectories(exp3Path, 'exp3Path')
countDirectories(exp4Path, 'exp4Path')
countDirectories(exp5Path, 'exp5Path')

countDirectories(exp6Path, 'exp6Path')
countDirectories(exp7Path, 'exp7Path')

#save on sheet
worksheet1 = spreedSheet.worksheet('Sheet1')
set_with_dataframe(worksheet1, df) 

In [ ]:
def makeModel(path, featureExtraction, classifier):
  raw_audio_train = createRawAudio(path + 'output/train/')
  raw_audio_test = createRawAudio(path + 'output/val/')

  X_train, y_train = audioFeatureExtraction(raw_audio_train, featureExtraction)
  X_test, y_test = audioFeatureExtraction(raw_audio_test, featureExtraction)

  ## X_train, X_test, y_train, y_test = trainTestSplit(X)
  model = get_scores(classifier, X_train, X_test, y_train, y_test)
  return (model)

In [ ]:
model = makeModel(originDataPath, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model, 'originData,spectralCentroid,RandomForestClassifier')


/content/drive/MyDrive/CryCareDataset/Dataset/output/train/
/content/drive/MyDrive/CryCareDataset/Dataset/output/val/


In [ ]:
model = makeModel(generatedData, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model, 'generatedData,SC,Svc')

/content/drive/MyDrive/CryCareDataset/generatedData/output/train/
/content/drive/MyDrive/CryCareDataset/generatedData/output/val/
file exist


In [ ]:
model2 = makeModel(generatedData, librosa.feature.mfcc, SVC)
exportClassificationReport(model2, 'generatedData,mfcc,SVC')

/content/drive/MyDrive/CryCareDataset/generatedData/output/train/
/content/drive/MyDrive/CryCareDataset/generatedData/output/val/


In [ ]:
model3 = makeModel(generatedData2, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model3, 'generatedData2,SC,RF')

/content/drive/MyDrive/CryCareDataset/generatedData2/output/train/
/content/drive/MyDrive/CryCareDataset/generatedData2/output/val/
file exist


In [ ]:
model3 = makeModel(exp4Path, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model3, 'exp8_pitch_1,SC,RF')

/content/drive/MyDrive/CryCareDataset/generatedData/Exp4/output/train/
/content/drive/MyDrive/CryCareDataset/generatedData/Exp4/output/val/


In [ ]:
model4 = makeModel(generatedData2, librosa.feature.mfcc, RandomForestClassifier)
exportClassificationReport(model4, 'generatedData2,mfcc,RF')

/content/drive/MyDrive/CryCareDataset/generatedData2/output/train/
/content/drive/MyDrive/CryCareDataset/generatedData2/output/val/


In [ ]:
model5 = makeModel(exp1Path, librosa.feature.mfcc, RandomForestClassifier)
exportClassificationReport(model5, 'exp1_speed_0.65,mfcc,RF')

/content/drive/MyDrive/CryCareDataset/generatedData/Exp1/output/train/
/content/drive/MyDrive/CryCareDataset/generatedData/Exp1/output/val/


In [ ]:
model9 = makeModel(onSilencePath, librosa.feature.chroma_stft, RandomForestClassifier)
exportClassificationReport(model9, 'onSilence,chroma_stft,RandomForestClassifier')

In [ ]:
model10 = makeModel(onSilencePath, librosa.feature.chroma_stft, SVC)
exportClassificationReport(model10, 'onSilence,chroma_stft,SVC')